In [97]:
import numpy as np
import pandas as pd

In [98]:
# Load in each dataset
data21 = pd.read_excel("Data/2021 Enrollment Demographics.xlsx", sheet_name="By School")
data22 = pd.read_excel("Data/2022 Enrollment Demographics.xlsx", sheet_name="By School")
data23 = pd.read_excel("Data/2021 Enrollment Demographics.xlsx", sheet_name="By School")

In [255]:
nameIndex = pd.read_excel("Data/Another School List.xlsx", sheet_name="School")
# Some schools have closed, so it is necessary to include them
closed = pd.read_excel("Data/Another School List.xlsx", sheet_name="Closed Schools")

# The CompositeSchoolID is composed of the LEA ID, the School ID, and the School Number
nameIndex["CompositeSchoolID"] = nameIndex["LEA ID"].astype(str) + nameIndex["School ID"].astype(str) + nameIndex["School Number"].astype(str)
closed["CompositeSchoolID"] = closed["LEA ID"].astype(str) + closed["School ID"].astype(str) + closed["School Number"].astype(str)
nameIndex = nameIndex[["LEA Name", "School Name", "CompositeSchoolID"]]
closed = closed[["LEA Name", "School Name", "CompositeSchoolID"]]

nameIndex = pd.concat([nameIndex, closed])

# There are a few leftover schools after matching. The dataset will be altered to match those
changeDict = {
    "Cache Alternative High": "Cache High",
    "Big Water School": "Big Water High",
    "Pleasant Creek School": "Pleasant Creek High School",
    "Ashley Valley Education Ctr.": "Ashley Valley Educ. Ctr.",
    "InTech Collegiate Academy": "Intech Collegiate Academy",
    "No. UT. Acad. of Math Engineering & Science - Ogden": "No. UT. Acad. of Math Engineering & Science Ogden",
    "Salt Lake Academy High School": "Real Salt Lake Academy High School",
    "Spectrum Academy": "Spectrum Academy - NSL",
    "Success Academy": "Success SUU"
}
nameIndex["School Name"] = nameIndex["School Name"].replace(changeDict)

nameIndex

,LEA Name,School Name,CompositeSchoolID
0,A to Z Building Blocks,A to Z Building Blocks American Fork on 100 East,186651186697901
1,A to Z Building Blocks,A to Z Building Blocks American Fork on 300 East,186651186698902
2,A to Z Building Blocks,A to Z Building Blocks Draper,186651186699903
3,A to Z Building Blocks,A to Z Building Blocks in Spanish Fork,186651186727906
4,A to Z Building Blocks,A to Z Building Blocks on Ranches Parkway,186651186700904
...,...,...,...
423,Weber State University Charter Academy,Weber State University Charter Academy,186314186315100
424,Whitmore Academy,Whitmore Academy,145604145604951
425,Willow Creek School,Willow Creek School,186003186003952
426,Winter Sports School,Winter Sports School,186187186187999


In [265]:
def CleanEnrollmentDemographics(data):
    """Cleans the dataset"""
    """Parameters: data (pandas dataframe)"""
    """Returns: cleaned (pandas dataframe)"""
    cleaned = data.copy()
    
    # Drop all the non-high schools.
    cleaned = cleaned[cleaned['Grade_12'] > 0]
    cleaned = cleaned[cleaned['Grade_11'] > 0]
    
    # Drop irrelevant columns
    irrelevantColumns = ["Grade_" + str(i) for i in range(1,9)] # I'm choosing to include grade 9, for now.
    irrelevantColumns += ["K", "Preschool"]
    cleaned = cleaned.drop(irrelevantColumns, axis="columns")
    
    # Merge to add the composite school id
    cleaned = cleaned.merge(nameIndex, how="left", on="School Name")
    
    return cleaned

In [267]:
CleanEnrollmentDemographics(data21)

,School Year,LEA TYPE,LEA Name_x,School Name,Total K-12,Grade_9,Grade_10,Grade_11,Grade_12,Female,...,Hispanic,Multiple Race,Pacific Islander,White,Economically Disadvantaged,English Learner,Student With a Disability,Homeless,LEA Name_y,CompositeSchoolID
0,2021,District,Alpine District,American Fork High,2449,7,824,810,808,1208.0,...,204.0,83.0,24.0,2098.0,266.0,29.0,160.0,1.0,Alpine District,122180704
1,2021,District,Alpine District,Cedar Valley High,2347,656,627,605,456,1148.0,...,280.0,94.0,26.0,1913.0,388.0,47.0,214.0,14.0,Alpine District,122186536721
2,2021,District,Alpine District,Dan W. Peterson,222,8,9,11,152,87.0,...,29.0,11.0,4.0,167.0,68.0,12.0,222.0,1.0,Alpine District,122177840
3,2021,District,Alpine District,Dan W. Peterson,222,8,9,11,152,87.0,...,29.0,11.0,4.0,167.0,68.0,12.0,222.0,1.0,Alpine District,122177640
4,2021,District,Alpine District,Horizon School,116,5,4,4,76,44.0,...,10.0,2.0,2.0,95.0,20.0,3.0,116.0,0.0,Alpine District,122186261808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,2021,Charter,Vanguard Academy,Vanguard Academy,463,81,82,88,73,211.0,...,1.0,0.0,0.0,462.0,321.0,0.0,20.0,0.0,Vanguard Academy,186380186381700
233,2021,Charter,Venture Academy,Venture Academy,751,93,103,66,72,379.0,...,101.0,3.0,6.0,627.0,178.0,17.0,127.0,0.0,Venture Academy,185874185874100
234,2021,Charter,Walden School of Liberal Arts,Walden School of Liberal Arts,443,39,38,30,34,215.0,...,59.0,16.0,0.0,358.0,187.0,15.0,92.0,3.0,Walden School of Liberal Arts,125582125582300
235,2021,Charter,Winter Sports School,Winter Sports School,109,29,27,27,26,47.0,...,3.0,2.0,0.0,102.0,0.0,0.0,0.0,0.0,Winter Sports School,186311186313700


In [268]:
CleanEnrollmentDemographics(data22)

,School Year,LEA TYPE,LEA Name_x,School Name,Total K-12,Grade_9,Grade_10,Grade_11,Grade_12,Female,...,Hispanic,Multiple Race,Pacific Islander,White,Economically Disadvantaged,English Learner,Student With a Disability,Homeless,LEA Name_y,CompositeSchoolID
0,2022,District,Alpine District,American Fork High,2475,75,795,819,786,1206.0,...,223.0,95.0,26.0,2093.0,279.0,25.0,176.0,2.0,Alpine District,122180704
1,2022,District,Alpine District,Cedar Valley High,2683,757,714,627,583,1314.0,...,375.0,122.0,41.0,2107.0,516.0,60.0,256.0,9.0,Alpine District,122186536721
2,2022,District,Alpine District,Dan W. Peterson,221,8,7,9,159,77.0,...,33.0,5.0,8.0,164.0,77.0,12.0,221.0,1.0,Alpine District,122177840
3,2022,District,Alpine District,Dan W. Peterson,221,8,7,9,159,77.0,...,33.0,5.0,8.0,164.0,77.0,12.0,221.0,1.0,Alpine District,122177640
4,2022,District,Alpine District,Horizon School,134,5,4,3,85,52.0,...,16.0,3.0,3.0,106.0,27.0,6.0,134.0,0.0,Alpine District,122186261808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,2022,Charter,Vanguard Academy,Vanguard Academy,507,80,77,78,71,241.0,...,7.0,2.0,0.0,498.0,337.0,0.0,21.0,0.0,Vanguard Academy,186380186381700
234,2022,Charter,Venture Academy,Venture Academy,755,98,87,80,61,371.0,...,124.0,4.0,4.0,618.0,136.0,23.0,140.0,0.0,Venture Academy,185874185874100
235,2022,Charter,Walden School of Liberal Arts,Walden School of Liberal Arts,418,37,35,39,26,218.0,...,53.0,14.0,0.0,342.0,172.0,14.0,89.0,0.0,Walden School of Liberal Arts,125582125582300
236,2022,Charter,Winter Sports School,Winter Sports School,111,29,28,27,27,46.0,...,3.0,2.0,0.0,105.0,0.0,0.0,3.0,0.0,Winter Sports School,186311186313700


In [269]:
CleanEnrollmentDemographics(data23)

,School Year,LEA TYPE,LEA Name_x,School Name,Total K-12,Grade_9,Grade_10,Grade_11,Grade_12,Female,...,Hispanic,Multiple Race,Pacific Islander,White,Economically Disadvantaged,English Learner,Student With a Disability,Homeless,LEA Name_y,CompositeSchoolID
0,2021,District,Alpine District,American Fork High,2449,7,824,810,808,1208.0,...,204.0,83.0,24.0,2098.0,266.0,29.0,160.0,1.0,Alpine District,122180704
1,2021,District,Alpine District,Cedar Valley High,2347,656,627,605,456,1148.0,...,280.0,94.0,26.0,1913.0,388.0,47.0,214.0,14.0,Alpine District,122186536721
2,2021,District,Alpine District,Dan W. Peterson,222,8,9,11,152,87.0,...,29.0,11.0,4.0,167.0,68.0,12.0,222.0,1.0,Alpine District,122177840
3,2021,District,Alpine District,Dan W. Peterson,222,8,9,11,152,87.0,...,29.0,11.0,4.0,167.0,68.0,12.0,222.0,1.0,Alpine District,122177640
4,2021,District,Alpine District,Horizon School,116,5,4,4,76,44.0,...,10.0,2.0,2.0,95.0,20.0,3.0,116.0,0.0,Alpine District,122186261808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,2021,Charter,Vanguard Academy,Vanguard Academy,463,81,82,88,73,211.0,...,1.0,0.0,0.0,462.0,321.0,0.0,20.0,0.0,Vanguard Academy,186380186381700
233,2021,Charter,Venture Academy,Venture Academy,751,93,103,66,72,379.0,...,101.0,3.0,6.0,627.0,178.0,17.0,127.0,0.0,Venture Academy,185874185874100
234,2021,Charter,Walden School of Liberal Arts,Walden School of Liberal Arts,443,39,38,30,34,215.0,...,59.0,16.0,0.0,358.0,187.0,15.0,92.0,3.0,Walden School of Liberal Arts,125582125582300
235,2021,Charter,Winter Sports School,Winter Sports School,109,29,27,27,26,47.0,...,3.0,2.0,0.0,102.0,0.0,0.0,0.0,0.0,Winter Sports School,186311186313700


In [ ]:
# TODO: Double-check those datasets (shouldn't they all be the same length?)
# TODO: Export to csv